In [1087]:
from pyraysum import prs, Geometry, Model, Control
import numpy as np
import os
import obspy as op
import pandas as pd
from scipy.optimize import dual_annealing, shgo, differential_evolution
from scipy import signal
import matplotlib.pyplot as plt
from scipy.optimize import LinearConstraint, Bounds
import ast

In [1088]:
# reading the data
path_to_data = "DATA/RF"
station = "FCC"
def reading_rfs(keyword: str, filters=(0.05, 0.3), corners = 4, sort=None, t_snr_treshold=-10, path_to_data="DATA/waveforms_list.csv") -> tuple:
    """
    Reading RF data by the name of keyword as station code
    inputs:
    keyword: str
        station code
    t_snr_treshold: float
        signal to noise ratio treshold for transverse component (default 0, means no treshold)

    outputs:
    obser: np.array
        observed data in numpy array format
    baz: list
        corresponding back azimuth
    slow: list
        corresponding slowness
    """
    baz = []
    slow = []
    waveforms_list = pd.read_csv("DATA/waveforms_list.csv")
    filtered_files = waveforms_list[waveforms_list['sta_code']==keyword].copy()
    filtered_files = filtered_files[filtered_files['rf_quality'] == 1].copy()
    path = "DATA/RF/"
    npts = 426 * 2   
    dt = 0.2
    obser = np.zeros((len(filtered_files), npts))
    
    # function to calculate signal to noise ratio for transverse component
    def cal_snr_for_transverse(rft: np.array) -> np.array:
        noise = rft[:213]
        signal = rft[213:]
        return np.sum(np.square(signal)) / np.sum(np.square(noise))
    
    idx = 0
    for _, row in filtered_files.iterrows():
        wave_path = os.path.join(path, row['file_name']+".pkl")
        st = op.read(wave_path)
        RFR = st.select(channel="RFR")[0].filter('bandpass', freqmin=filters[0], freqmax=filters[1], corners=corners, zerophase=True).data
        RFT = st.select(channel="RFT")[0].filter('bandpass', freqmin=filters[0], freqmax=filters[1], corners=corners, zerophase=True).data
        #check if the signal to noise ratio is above the treshold
        if cal_snr_for_transverse(RFT) > t_snr_treshold:
            baz.append(st[0].stats.baz)
            slow.append(st[0].stats.slow)
            RFR = RFR / np.max(np.abs(RFR))
            RFT = RFT / np.max(np.abs(RFT))
            obser[idx, :426] = RFR
            obser[idx, 426:] = RFT
            idx += 1
    obser = obser[:idx, :]
    # sorting based on back azimuth or slowness
    if sort != None:
        zipped = list(zip(obser, baz, slow))
        if sort == "baz":
            zipped.sort(key=lambda x: x[1])
        elif sort == "slow":
            zipped.sort(key=lambda x: x[2])
        obser, baz, slow = zip(*zipped)
    obser = np.array(obser)
    return obser, baz, slow



# reading the model
def read_model(path_to_models, layer):
    """
    Reading the initials bounds and values
    Inputs:
        path_to_models: str: path to the models
        layer: int: layer number
    Outputs:
        bounds: list: bounds of the model
        fixed_vales: list: fixed values of the model
        mask: np.array: mask for the non-fixed values
    """
    bounds = pd.read_csv(os.path.join(path_to_models, "bounds.csv"))
    fixed_vales = pd.read_csv(os.path.join(path_to_models, "fixed_values.csv"))
    bounds = bounds[bounds["layer_code"] == layer].drop(columns=["layer_code"])
    fixed_vales = fixed_vales[fixed_vales["layer_code"] == layer].drop(columns=["layer_code"])
    
    #fix a bug
    fixed_vales.loc[len(fixed_vales.ani)-1, "ani"]=np.nan
    
    bounds = bounds.values.flatten()
    # create a mask for the non-fixed values
    mask = pd.notna(bounds)
    bounds = [ast.literal_eval(i) for i in bounds if str(i) != 'nan']
    return bounds, fixed_vales, mask

def pyraysum_func(baz, slow, thickn, rho, vp, vs, dip, strike, plunge, trend, ani, npts, dt) -> tuple:
    model = Model(thickn, rho, vp, vs=vs, strike=strike, dip=dip, plunge=plunge, trend=trend, ani=ani)
    geom = Geometry(baz, slow)
    rc = Control(wvtype="P", rot="RTZ", mults=2, verbose=False, npts=npts*1, dt=dt, align=1, shift=5)
    result = prs.run(model, geom, rc, rf=True)
    result.filter('rfs', 'lowpass', freq=1., zerophase=True, corners=2)
    return result

def modeling(model: pd.DataFrame, baz, slow, filters=(0.05, 0.3), corners=4):
    result = pyraysum_func(baz, slow, model["thickn"], model["rho"], model["vp"], model["vs"], model["dip"],
                                     model["strike"], model["plunge"], model["trend"], model["ani"], 3*426, 0.2)
    #lower number of samples would be problematic and that is why we produce longer signal and then reduce it
    pred = np.zeros((len(result), 2*426)) 
    pred_r = np.zeros((len(result), 3*426))
    pred_t = np.zeros((len(result), 3*426))

    for idx, i in enumerate(result):
        RFR = i[1][0].filter('bandpass', freqmin=filters[0], freqmax=filters[1], corners=corners, zerophase=True).data
        RFT = i[1][1].filter('bandpass', freqmin=filters[0], freqmax=filters[1], corners=corners, zerophase=True).data
        pred_r[idx, :] = RFR / np.max(np.abs(RFR))
        pred_t[idx, :] = RFT / np.max(np.abs(RFT))
    #making more samples to reduce the effect of the edges
    pred_r = pred_r[:, 426:2*426]
    pred_t = pred_t[:, 426:2*426]
    pred[:, :426], pred[:, 426:] = pred_r, pred_t
        #reducing the length of the data to 426
    return pred

def cost_func(variables, fixed_values, mask, obser, baz, slow, layer):
    """
    """
    fixed_values = fixed_values.values.flatten()
    fixed_values[mask] = variables
    model = pd.DataFrame(fixed_values.reshape(-1, len(fixed_values)), columns=fixed_values.index)
    pred = modeling(model, baz, slow)
    loss = np.sum(np.square(obser - pred))
    return loss  
    



In [1089]:
#Control 
filters = (0.05, 0.3)
corners = 4
#reading RFs
path_to_data = "DATA/RF"
station = "BLKN"
obser, baz, slow = reading_rfs(station, t_snr_treshold=4, sort="baz", filters=filters, corners=corners)
print("Station:", station,"- Observation Data Size:", obser.shape)
# reading the model bounds and fixed values
path_to_models = "inv/initial"
layer = 2
bounds, fixed_values, mask = read_model(path_to_models, layer)
mask

Station: BLKN - Observation Data Size: (8, 852)


array([ True,  True,  True,  True, False, False,  True,  True,  True,
       False, False, False, False,  True,  True,  True,  True,  True])

In [1090]:
mask = pd.isna(fixed_values)
fixed_values[mask] = 5000
print(fixed_values)

# modeling the data
results = modeling(fixed_values, baz, slow, filters=filters, corners=corners)

   thickn     rho      vp      vs     dip  strike  plunge   trend   ani
0  5000.0  5000.0  5000.0  5000.0     0.0     0.0  5000.0  5000.0  5000
1     0.0  3500.0  8100.0  3500.0  5000.0  5000.0  5000.0  5000.0  5000
